In [ ]:
# z výstupu z wikidata.ipynb přeměňujeme názvy kategorií na názvy článku a posléze LAU ID obcí
# nejčastější případ: kategorie Narození v (obci) má jednu nadkategorii neobsahující "Narození...", identickou se stránkou obce
# ze stránky obce ID
from bs4 import BeautifulSoup
import json

In [ ]:
# requesty na obecné wikipedia API
import requests
session = requests.Session()
def api_request(request):
    api_url = "https://cs.wikipedia.org/w/api.php"
    header = {"Api-User-Agent": "michal.zlatkovsky@rozhlas.cz"}
    reply = session.get(url=api_url, headers=header, params=request)
    return reply.json()

In [ ]:
def get_obec_pagename(cat_data):
    try:
        # defaultní případ: obec má Narození.../Kategorie k údržbě, nemá Osobnosti => zbývá jméno obce v kat
        cats = [entry["*"] for entry in cat_data["parse"]["categories"]]
        filtered_cats = [cat for cat in cats if "Narození" not in cat and "Osobnosti" not in cat and "Lidé" not in cat and "Údržba" not in cat]
        #Osobnosti z..., Lidé z...
        if len(filtered_cats) == 0:
            filtered_cats = [cat for cat in cats if "Osobnosti" in cat or "Lidé" in cat]
            nadkat = filtered_cats[0]
            request = {"action": "parse", "format": "json", "page": "Kategorie:"+nadkat, "prop": "categories"}
            cats = [entry["*"] for entry in api_request(request)["parse"]["categories"]]
            filtered_cats = [cat for cat in cats if "Narození" not in cat and "Osobnosti" not in cat and "Lidé" not in cat and "Údržba" not in cat]
        if len(filtered_cats) == 1: return filtered_cats[0]
        else: return False
    except:
        return False

In [ ]:
with open("filtered_data.json", encoding="utf8") as file:
    data = json.loads(file.read())

In [ ]:
flat_data = {}
# flattening dat
for kraj in data:
    flat_data.update(data[kraj])

In [ ]:
# kategorie - prvni pass
for cat in flat_data:
    if len(flat_data[cat]) == 2:
        request = {
            "action": "parse",
            "format": "json",
            "page": cat,
            "prop": "categories"
        }
        cat_data = api_request(request)
        flat_data[cat].append(get_obec_pagename(cat_data))

In [ ]:
# uložení všech dat před manuálním dočištěním
with open("ident_data_first_pass.json", "w", encoding="utf8") as file:
    file.write(json.dumps(flat_data))

In [ ]:
# fix chyb
flat_data["Kategorie:Narození v Praze"][2] = "Praha"
flat_data["Kategorie:Narození v Mostě"][2] = "Most (město)"
flat_data["Kategorie:Narození v Kravařích"][2] = "Kravaře"
flat_data["Kategorie:Narození v Bílině"][2] = "Bílina (město)"
flat_data["Kategorie:Narození v Žebráku"][2] = "Žebrák (okres Beroun)"

In [ ]:
# scrape ID obci
request["prop"] = "text"
for cat in flat_data:
    if len(flat_data[cat]) == 3 and flat_data[cat][2] != False:
        try:
            request["page"] = flat_data[cat][2]
            data = api_request(request)
            soup = BeautifulSoup(data["parse"]["text"]["*"], 'html.parser')
            try:
                obec_id = soup.find(title="Local administrative unit").parent.parent.td.text.split()[1]
            except:
                obec_id = soup.find(title="CZ-NUTS").parent.parent.td.text.split()[1]
            print(flat_data[cat], obec_id)
            flat_data[cat].append(obec_id)
        except:
            print(cat, "ERROR!", flat_data[cat])

In [ ]:
# fix scrape chyby u Prahy
flat_data["Kategorie:Narození v Praze"][3] = "554782"

In [ ]:
# finalni organizace dat
final_data = []
for obec in flat_data.values():
    if len(obec) == 4:
        final_data.append({"obec": obec[2].replace("_"," "), "id": int(obec[3]), "rodak": obec[0], "pv": obec[1]})

In [348]:
with open("final_data.json", "w", encoding="utf8") as file:
    file.write(json.dumps(final_data))